# Aging Contact Surface

Predicting age of contact from contact interface gray-scale images.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from __future__ import absolute_import, division, print_function
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Get 3D matrix of data
data = []
labels = []
for i in range(500):
    high_t.append(np.loadtxt('./PS2-a-highT/'+str(i).zfill(3), delimiter=','))

In [ ]:
# Get data in right shape
labels = np.array(labels)
data = np.dstack(data).T

# Shuffle data and labels
s = np.arange(data.shape[0])
np.random.shuffle(s)
threeD_data = data[s,:,:]
labels = labels[s]

# Split train and test
train_data = data[:800,:,:]
test_data = data[800:,:,:]
train_labels = labels[:800]
test_labels = labels[800:]

In [ ]:
# Build model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(64,64)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

# Compile model
model.compile(optimizer='sgd', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels, epochs=5)
predictions=model.predict_proba(test_data)

# Make predictions
sq_pred_temps = []
for test_img in predictions:
    weighted_pred = np.dot(Class_Times, test_img)
    sq_pred_temps.append(weighted_pred)